# Customer Churn Analysis

In [1]:
import pandas as pd
data = pd.read_csv('Customer_churn_analysis.csv')

## Dataset Overview

In [2]:
data.shape[0]

10000

In [3]:
data.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [5]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## Data Cleaning an Preprocessing 

*Removing Duplicated Values*

In [6]:
data.duplicated().sum()

np.int64(0)

*Handling Null Values*

In [7]:
data.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

*Data Type Conversion*

In [8]:
data.dtypes

RowNumber            int64
CustomerId           int64
Surname             object
CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

In [9]:
data['CustomerId'] = data['CustomerId'].astype(str)
data['CustomerId'].dtypes

dtype('O')

## Data Aggregation

In [10]:
customer_by_geography = data.groupby('Geography')['CustomerId'].count()
customer_by_geography

Geography
France     5014
Germany    2509
Spain      2477
Name: CustomerId, dtype: int64

In [11]:
data['CreditScoreCategory'] = pd.cut(data['CreditScore'], bins=[300, 500, 650, 800, 900], labels=['Poor', 'Fair', 'Good', 'Excellent'])
data['CreditScoreCategory'].unique()

['Fair', 'Good', 'Excellent', 'Poor']
Categories (4, object): ['Poor' < 'Fair' < 'Good' < 'Excellent']

In [12]:
credit_score_analysis = data.groupby('CreditScoreCategory')['Exited'].mean().reset_index()
credit_score_analysis

C:\Users\Pavan Gowda R\AppData\Local\Temp\ipykernel_15992\2608549936.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  credit_score_analysis = data.groupby('CreditScoreCategory')['Exited'].mean().reset_index()


,CreditScoreCategory,Exited
0,Poor,0.236392
1,Fair,0.210759
2,Good,0.193074
3,Excellent,0.196899


In [13]:
pivot_churn = data[data['Exited']==1].pivot_table(values='Exited', index='Geography', columns='NumOfProducts', aggfunc= 'count').round(2)
pivot_churn

NumOfProducts,1,2,3,4
Geography,,,,
France,564,135,82,29
Germany,578,126,86,24
Spain,267,87,52,7


## Key Metrics

*1. Customer & Retention Metrics*

- *Churn Rate = (Exited Customers / Total Customers) × 100*

In [14]:
exited_customers = data['Exited'].eq(1).sum()
total_custommers = data['CustomerId'].count()
churn_rate = ( exited_customers / total_custommers) * 100
churn_rate.round(2)

np.float64(20.37)

- *Retention Rate = (1 - Churn Rate) × 100*

In [15]:
retention_rate = (1-churn_rate) * 100
retention_rate.round()

np.float64(-1937.0)

- *Average Tenure of Churned Customers*

In [16]:
average_tenure = data['Tenure'].mean().round()
average_tenure

np.float64(5.0)

- *Top Churned Customer Segments (based on Geography, Age, Balance, etc.)*

In [17]:
geography_churn = data[data['Exited']==1].groupby('Geography')['Exited'].count()
geography_churn

Geography
France     810
Germany    814
Spain      413
Name: Exited, dtype: int64

In [18]:
data['Age Category'] = pd.cut(data['Age'], bins=[18,30,50,70,90], labels=['18-29','30-49','50-69','70-89'], right=False)
age_churn = data[data['Exited']==1].groupby('Age Category')['Exited'].count()
age_churn

C:\Users\Pavan Gowda R\AppData\Local\Temp\ipykernel_15992\802312071.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  age_churn = data[data['Exited']==1].groupby('Age Category')['Exited'].count()


Age Category
18-29     124
30-49    1279
50-69     619
70-89      15
Name: Exited, dtype: int64

*2. Customer Segmentation & behavior*

- *High-Value vs. Low-Value Customers*

- *Churn by Product Usage (Do customers with multiple products churn less?)*

In [19]:
churn_by_products = data.groupby('NumOfProducts')['Exited'].mean()*100
churn_by_products.round(2)

NumOfProducts
1     27.71
2      7.58
3     82.71
4    100.00
Name: Exited, dtype: float64

- *Active vs. Inactive Customers (How does inactivity impact churn?)*

In [20]:
##  There are no inactive customers

*3. Geographical Churn Trends*

- *Churn Rate per Country = (Churned Customers in a Country / Total Customers in that Country)*

In [21]:
geography_churn

Geography
France     810
Germany    814
Spain      413
Name: Exited, dtype: int64

- *Regional Retention Strategies (Country-specific campaigns)*

*4. Financial & Credit Analysis*

- *Churn Rate vs. Balance (Does having a higher balance reduce churn risk?)*

In [22]:
data['Balance Category'] = pd.cut(
    data['Balance'],
    bins=[-1, 0, 50000, 100000, 150000, 200000, data['Balance'].max()],
    labels=['Zero', 'Low (1-50K)', 'Medium (50K-100K)', 'High (100K-150K)', 'Very High (150K-200K)', 'Ultra High']
)

churn_by_balance = data.groupby('Balance Category')['Exited'].mean() * 100
churn_by_balance


C:\Users\Pavan Gowda R\AppData\Local\Temp\ipykernel_15992\3281973230.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  churn_by_balance = data.groupby('Balance Category')['Exited'].mean() * 100


Balance Category
Zero                     13.823611
Low (1-50K)              34.666667
Medium (50K-100K)        19.880716
High (100K-150K)         25.770235
Very High (150K-200K)    21.925134
Ultra High               55.882353
Name: Exited, dtype: float64

- *Credit Score Distribution (Do low-credit customers churn more?)*

In [23]:
data['Credit Score Category'] = pd.cut(
    data['CreditScore'],
    bins=[300, 500, 600, 700, 800, 900, 1000],
    labels=['Very Poor (300-499)', 'Poor (500-599)', 'Fair (600-699)', 'Good (700-799)', 'Very Good (800-899)', 'Excellent (900-1000)']
)
churn_by_credit = data.groupby('Credit Score Category')['Exited'].mean() * 100
churn_by_credit.round(2)

C:\Users\Pavan Gowda R\AppData\Local\Temp\ipykernel_15992\574346626.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  churn_by_credit = data.groupby('Credit Score Category')['Exited'].mean() * 100


Credit Score Category
Very Poor (300-499)     23.64
Poor (500-599)          21.17
Fair (600-699)          19.72
Good (700-799)          19.91
Very Good (800-899)     19.69
Excellent (900-1000)      NaN
Name: Exited, dtype: float64